In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network
from sklearn.preprocessing import MinMaxScaler

In [2]:
# import edgelist data
entity_wel_df = pd.read_csv('entity_weighted_edgelist_ML.csv', index_col=0)
entity_wel_df['ent_rel_pair'] = entity_wel_df['head_text'] +' '+entity_wel_df['relation']+' '+entity_wel_df['tail_text']
entity_wel_df['head_name'] = entity_wel_df['head_text']+'_'+entity_wel_df['head_wiki_id']
entity_wel_df['tail_name'] = entity_wel_df['tail_text']+'_'+entity_wel_df['tail_wiki_id']

# node size
head_node_weight_df = entity_wel_df.groupby(['head_name']).sum().sort_values('count', ascending=False).reset_index()
tail_node_weight_df = entity_wel_df.groupby(['tail_name']).sum().sort_values('count', ascending=False).reset_index()
head_node_weight_df = entity_wel_df.merge(head_node_weight_df, left_on=['head_name'], right_on=['head_name'], how='right')
tail_node_weight_df = entity_wel_df.merge(tail_node_weight_df, left_on=['tail_name'], right_on=['tail_name'], how='right')
head_node_weight_df['node'] = head_node_weight_df['head_text']
head_node_weight_df['size'] = head_node_weight_df['count_y']
head_node_weight_df = head_node_weight_df[['size','node']]
tail_node_weight_df['node'] = tail_node_weight_df['tail_text']
tail_node_weight_df['size'] = tail_node_weight_df['count_y']
tail_node_weight_df = tail_node_weight_df[['size','node']]
node_weight_df = pd.concat([head_node_weight_df, tail_node_weight_df])
node_weight_df['size'] = MinMaxScaler(feature_range=(1, 10)).fit_transform(np.array(node_weight_df['size']).reshape(-1,1))
node_weight_df

,size,node
0,3.720232,machine learning
1,3.720232,machine learning
2,3.720232,machine learning
3,3.720232,machine learning
4,3.720232,machine learning
...,...,...
75810,1.000000,concrete quality
75811,1.000000,concrete design
75812,1.000000,concrete decisions
75813,1.000000,concordance


In [3]:
# df = entity_wel_df.iloc[:1000]
mention_num = 2
entity_wel_df = entity_wel_df[entity_wel_df['head_text'] != 'Elsevier'] # df = df[~df[col].isin([string])]
entity_wel_df = entity_wel_df[entity_wel_df['tail_text'] != 'Elsevier']
df = entity_wel_df[entity_wel_df['count']>mention_num]
df.rename(columns={"count": "value", "relation":"title"}, inplace=True)
node_lst = df['head_text'].tolist()+df['tail_text'].tolist()
node_weight_df = node_weight_df.drop_duplicates(subset='node').reset_index(drop=True)
node_weight_df = node_weight_df[node_weight_df['node'].isin(node_lst)]
node_attr = node_weight_df.set_index('node').to_dict('index')
G = nx.from_pandas_edgelist(df, source='head_text', target='tail_text', edge_attr=['value', 'title'])
nx.set_node_attributes(G, node_attr)

nt = Network(height="750px", width="100%", notebook=True,directed=True)
# populates the nodes and edges data structures
nt.from_nx(G)
nt.toggle_physics(True)
nt.show('ML_above2edges.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


C:\Users\gcmar\AppData\Local\Temp\ipykernel_88984\1822068212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)
